# <span style="color:turquoise">Text classification with pytorch</span>


An example of using natural language processing for sentiment analysis. <br> Building a binary classifier of movie reviews that will predict if a review is positive or negative.




__Dataset:__ IMDB movie reviews from Kaggle<br>
__Model:__ bag-of-words + RNN(?)


### <span style="color:teal">Todo:</span>

- ~~Read dataset~~
- ~~Preprocess text~~
- ~~Split into train, validation, and test sets~~
- Vectorize text
- Make model
- Make training function
- Make evaluation function
- Train
- Evaluate

In [20]:
import torch
import csv
import random

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


## <span style="color:teal">Read the data and split it into training, cross-validation, and test sets</span>

In [21]:
class Reviews():
    
    def __init__(self):
        self.train = {}
        self.val = {}
        self.test = {}
        self.LABELS = {"positive":1, "negative": 0}
        self.COUNT = {"positive": 0, "negative": 0}
    
    
    def read_data(self):
        
        dataset = []
        
        with open ("IMDB_Dataset.csv", newline='') as f:
            datareader = csv.reader(f, delimiter=',')
            next(datareader, None)

            for row in datareader:
                dataset.append([row[0], self.LABELS[row[1]]])
                self.COUNT[row[1]] += 1
                
        
        
        random.shuffle(dataset)
            
                
        return dataset




    def split_dataset(self,
                      dataset,
                      split=[int(50000*0.6), int(50000*0.2), int(50000*0.2)]):
        
        result = torch.utils.data.random_split(dataset,
                                               split,
                                               generator=torch.Generator().manual_seed(42))
        #for i in range(len(dataset)):
            #self.data[i] = {'text': dataset[i][0], 'label':dataset[i][1]}  
            
        return result

In [22]:
rev = Reviews()
data = rev.read_data()
pos_count = rev.COUNT["positive"]
neg_count = rev.COUNT["negative"]


In [23]:
train, val, test = rev.split_dataset(data)
print(train, val, test)

<torch.utils.data.dataset.Subset object at 0x7efdbcd63790> <torch.utils.data.dataset.Subset object at 0x7efdbcd637f0> <torch.utils.data.dataset.Subset object at 0x7efdbcd631f0>


In [24]:
print(len(train), len(val), len(test))

30000 10000 10000


## <span style="color:teal">Preprocess data</span>

In [25]:
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

import re

In [54]:
def preprocess(review,
               remove_stopwords=False, 
               remove_html=True, 
               remove_punct=False, 
               lowercase=False, 
               lemmatize=False):
    
    if remove_html:
        review = re.sub(r'<.*>', ' ', review)
    
    review = word_tokenize(review)
        
    if remove_stopwords:
        stop_words = set(stopwords.words("english"))
        review = [w for w in review if w not in stop_words]
        
    if remove_punct:
        contractions = ["'ll", "'s", "n't", "'d", "'m", "'ve", "'re"]
        review = [w for w in review if w.isalnum() or w in contractions]
    
    if lowercase:
        review = [w.lower() for w in review]
        
    if lemmatize:
        lemmatizer = WordNetLemmatizer()
        review = [lemmatizer.lemmatize(w) for w in review]
    
    
    return review
    


## <span style="color:teal">Vectorize the data as a continuous bag-of-words</span>

In [39]:
def make_vocabulary_dicts(data):
    vocab = set()
    for review, _ in data:
        words = preprocess(review, lowercase=True, remove_punct=True)
        for word in words:
            vocab.add(word)
            
    vocab_sorted = sorted(vocab)
    word2ind = {word : i for i, word in enumerate(vocab_sorted)}
    ind2word = {i : word for i, word in enumerate(vocab_sorted)}
    
    return word2ind, ind2word

In [71]:
word2ind, ind2word = make_vocabulary_dicts(data)
print(len(word2ind), len(ind2word))
print(word2ind['never'], word2ind['awful'])
print(ind2word[6700], ind2word[10582])

79436 79436
48367 5411
became cacoyannis
